## Parameters and initalization

In [28]:
n = 50 # Number of games selected to train on, higher is better but slower


# change working directory to the "laserforce_ranking" directory
import os
import IPython
notebook_name = "/".join(
        IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[-5:]
    )
os.chdir(os.path.dirname(notebook_name))
os.chdir("..")

from tortoise import Tortoise
from config import config
from db.config import get_tortoise_orm_config

TORTOISE_ORM = get_tortoise_orm_config(config)

await Tortoise.init(config=TORTOISE_ORM)


## Set ranges for paramaters

In [29]:
from helpers.ratinghelper import \
BETA, KAPPA, TAU, \
SM5_HIT_WEIGHT_MU, SM5_HIT_WEIGHT_SIGMA, SM5_HIT_MEDIC_WEIGHT_MU, SM5_HIT_MEDIC_WEIGHT_SIGMA, \
SM5_MISSILE_WEIGHT_MU, SM5_MISSILE_WEIGHT_SIGMA, SM5_MISSILE_MEDIC_WEIGHT_MU, SM5_MISSILE_MEDIC_WEIGHT_SIGMA, \
LB_STEAL_WEIGHT_MU, LB_STEAL_WEIGHT_SIGMA, LB_GOAL_WEIGHT_MU, LB_GOAL_WEIGHT_SIGMA, LB_ASSIST_WEIGHT_MU, LB_ASSIST_WEIGHT_SIGMA

# {"PARAM_NAME": (value, uncertainty), ...}

# uncertainty creates a range of possible values for the parameter
# (value - value * uncertainty, value + value * uncertainty)

PARAMS = {
    "BETA": (BETA, 1/2),
    "KAPPA": (KAPPA, 1/2),
    "TAU": (TAU, 1/2),
    "SM5_HIT_WEIGHT_MU": (SM5_HIT_WEIGHT_MU, 1/2),
    "SM5_HIT_WEIGHT_SIGMA": (SM5_HIT_WEIGHT_SIGMA, 1/2),
    "SM5_HIT_MEDIC_WEIGHT_MU": (SM5_HIT_MEDIC_WEIGHT_MU, 1/2),
    "SM5_HIT_MEDIC_WEIGHT_SIGMA": (SM5_HIT_MEDIC_WEIGHT_SIGMA, 1/2),
    "SM5_MISSILE_WEIGHT_MU": (SM5_MISSILE_WEIGHT_MU, 1/2),
    "SM5_MISSILE_WEIGHT_SIGMA": (SM5_MISSILE_WEIGHT_SIGMA, 1/2),
    "SM5_MISSILE_MEDIC_WEIGHT_MU": (SM5_MISSILE_MEDIC_WEIGHT_MU, 1/2),
    "SM5_MISSILE_MEDIC_WEIGHT_SIGMA": (SM5_MISSILE_MEDIC_WEIGHT_SIGMA, 1/2),
}

In [31]:
from db.sm5 import SM5Game
import random

async def get_highest_id():
    max_id_record = await SM5Game.all().order_by("-id").first()
    if max_id_record:
        highest_id = max_id_record.id
    else:
        highest_id = 1
    return highest_id

highest_id = await get_highest_id()

game_ids = random.sample(range(1, highest_id + 1), 50)

games = await SM5Game.filter(id__in=game_ids).all()

print(games)
print(f"Loaded {len(games)} games")

[SM5Game(id=2, start_time=2023-06-11 20:29:14+00:00, arena=4-43, mission_type=5, mission_name=Space Marines 5 Tournament Edition, winner=Team.GREEN, ranked=False), SM5Game(id=3, start_time=2023-06-11 20:33:46+00:00, arena=4-43, mission_type=5, mission_name=Space Marines 5 Tournament Edition, winner=Team.GREEN, ranked=False), SM5Game(id=6, start_time=2023-06-11 21:18:36+00:00, arena=4-43, mission_type=5, mission_name=Space Marines 5 Tournament Edition, winner=Team.RED, ranked=True), SM5Game(id=78, start_time=2024-01-14 19:39:13+00:00, arena=4-43, mission_type=5, mission_name=Space Marines 5 Tournament Edition, winner=Team.GREEN, ranked=True), SM5Game(id=98, start_time=2024-02-11 17:54:51+00:00, arena=4-43, mission_type=5, mission_name=Space Marines 5 Tournament Edition, winner=Team.RED, ranked=True), SM5Game(id=107, start_time=2024-02-18 19:57:03+00:00, arena=4-43, mission_type=5, mission_name=Space Marines 5 Tournament Edition, winner=Team.GREEN, ranked=True), SM5Game(id=135, start_tim

In [32]:
import numpy as np
from sklearn.metrics import log_loss

def is_draw_by_rule(p_pred, scoreA, scoreB, prob_margin=0.08, score_margin=3000):
    return (abs(p_pred - 0.5) < prob_margin) and (abs(scoreA - scoreB) <= score_margin)

def get_predicted_pA(playerA, playerB, params):
    # set params then use model

    return 0 # use model

def make_three_way_probs(p_A_win, draw_width=0.12):
    draw_prob = max(0.0, (draw_width - 2 * abs(p_A_win - 0.5)) / draw_width)
    pA = p_A_win * (1 - draw_prob)
    pB = (1 - p_A_win) * (1 - draw_prob)
    s = pB + draw_prob + pA
    return [pB / s, draw_prob / s, pA / s]

def fitness(params, draw_width=0.12, prob_margin=0.08, score_margin=3):
    y_true = []   # integers 0/1/2
    y_pred = []   # list of [P(B), P(draw), P(A)]
    for playerA, playerB, scoreA, scoreB, actual in games:
        pA = get_predicted_pA(playerA, playerB, params)

        # Option A: use rule to decide if the *observed* game is a draw
        # (if you prefer to infer draws from scores instead of stored label)
        observed_is_draw = is_draw_by_rule(pA, scoreA, scoreB, prob_margin, score_margin)

        # compute actual label
        if observed_is_draw:
            true_label = 1
        else:
            true_label = 2 if scoreA > scoreB else 0

        # predicted 3-class probability
        probs = make_three_way_probs(pA, draw_width=draw_width)

        y_true.append(true_label)
        y_pred.append(probs)

    # sklearn expects y_true as labels and y_pred as probs in shape (n_samples, n_classes)
    # Convert to one-hot? log_loss accepts labels as ints and probs as array.
    y_pred = np.array(y_pred)
    return log_loss(y_true, y_pred, labels=[0,1,2])

    
    

In [ ]:
from scipy.optimize import differential_evolution

# create bounds

bounds = []

for param_name, value_and_multiplier in PARAMS.items():
    value, multiplier = value_and_multiplier

    bounds.append((value - value * multiplier, value + value * multiplier))

result = differential_evolution(fitness, bounds, maxiter=50, seed=42)
print("Best params:", result.x)
print("Fitness:", result.fun)

Bounds: [(2.0833333333333335, 6.25), (5e-05, 0.00015000000000000001), (0.045454545454545456, 0.13636363636363635), (0.05, 0.15000000000000002), (0.05, 0.15000000000000002), (0.1, 0.30000000000000004), (0.05, 0.15000000000000002), (0.125, 0.375), (0.05, 0.15000000000000002), (0.25, 0.75), (0.05, 0.15000000000000002)]


RuntimeError: The map-like callable must be of the form f(func, iterable), returning a sequence of numbers the same length as 'iterable'